#### Import Modlues

In [9]:
import sys, os
import cohere
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from read_write_util import ReadWriteUtil
from prompt_pipeline import PromptPipeline

#### Read Cleaned Data

In [5]:
reader = ReadWriteUtil()
API_KEY = os.getenv('API_KEY')
# print(API_KEY)
#Initialize reader and API_KEY

In [6]:
train_data =  reader.dvc_get_data('../data/job_description_train_cleaned.json', 'jdtrain_v1_cleaned')
train_data[0]

2022-09-14 06:24:23,792:logger:../data/job_description_train_cleaned.json with version jdtrain_v1_cleaned Loaded


{'document': "Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience",
 'tokens': [{'text': 'Bachelor', 'entityLabel': 'DIPLOMA'},
  {'text': 'Mechanical Engineering', 'entityLabel': 'DIPLOMA_MAJOR'},
  {'text': 'Physical Science', 'entityLabel': 'DIPLOMA_MAJOR'},
  {'text': '3+ years', 'entityLabel': 'EXPERIENCE'},
  {'text': 'developing', 'entityLabel': 'SKILLS'},
 

In [7]:
test_data = reader.dvc_get_data('../data/job_description_test_cleaned.json', 'jdtest_v1_cleaned')
test_data[0]

2022-09-14 06:24:24,031:logger:../data/job_description_test_cleaned.json with version jdtest_v1_cleaned Loaded


{'document': '\nCurrently holding a faculty, industry, or government researcher position.\nPh.D. and publications in machine learning, AI, computer science, statistics, applied mathematics, data science, or related technical fields.\nExperience leading a team in solving analytical problems using quantitative approaches.\nExperience manipulating and analyzing data from different sources.\nExperience in theoretical and empirical research and for answering questions with research.\nAbility to communicate research for public audiences of peers.\nKnowledge in a programming language.\nAbility to obtain and maintain work authorization in the country of employment in 2018.\n\nPREFERRED \n1+ year(s) of work experience in a university, industry, or government lab(s), in a role with primary emphasis on AI research.\nExperience driving original scholarship in collaboration with a team.\nFirst-author publications at peer-reviewed AI conferences (e.g. NIPS, CVPR, ICML, ICLR, ICCV, and ACL).\nExperie

### Try Different Prompt Techniques

`We are going to use cohere to extract entities from the job description, we are going to use different techniques to come up with an effective and dynamic prompt`

#### Initialize our Cohere Client

In [10]:
co = cohere.Client(API_KEY)

#### Technique 1: Choosing the first two in our training json as a prompt to be used by cohere's LLM Generate

In [8]:
tech1_train_data = train_data[:2]
tech1_train_data
#Picking the first two arrays from our json

[{'document': "Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience",
  'tokens': [{'text': 'Bachelor', 'entityLabel': 'DIPLOMA'},
   {'text': 'Mechanical Engineering', 'entityLabel': 'DIPLOMA_MAJOR'},
   {'text': 'Physical Science', 'entityLabel': 'DIPLOMA_MAJOR'},
   {'text': '3+ years', 'entityLabel': 'EXPERIENCE'},
   {'text': 'developing', 'entityLabel': 'SKILL

#### Extract Values from json

In [11]:

response = co.generate( 
  model='xlarge', 
  prompt='Job Description: Bachelor\'s degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience\n\nDIPLOMA: Bachelor\nDIPLOMA_MAJOR: Mechanical Engineering, Physical Science\nEXPERIENCE: 3+ years\nSKILLS: developing, fiber optic cables, connector related products\n--\n', 
  max_tokens=400, 
  temperature=0.5, 
  k=0, 
  p=1, 
  frequency_penalty=0, 
  presence_penalty=0, 
  stop_sequences=["--"], 
  return_likelihoods='NONE') 
print('Prediction: {}'.format(response.generations[0].text))

Prediction: --
